For trending markets: Moving Averages + RSI + ADX
This is an excellent combination for confirming and entering a strong trend.
Moving Averages (MA): Identifies the general direction of the trend.
Usage: A shorter-term MA crossing above a longer-term MA (a "golden cross") can signal a new uptrend.
Relative Strength Index (RSI): Confirms momentum and helps avoid entering an already overextended market.
Usage: In an uptrend, confirm buying momentum with an RSI above 50 and rising. Avoid entering if the RSI is in the overbought zone (above 70).
Average Directional Index (ADX): Measures the strength of the trend, not the direction.
Usage: Look for a rising ADX above 25 to confirm that a trend has strength. 

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279221,85.324368,83.909697,84.556835,118944600
1,2010-01-05,85.504944,85.542570,84.918006,85.226528,111579900
2,2010-01-06,85.565170,85.775865,85.354475,85.422196,116074400
3,2010-01-07,85.926346,86.031693,85.166334,85.407129,131091100
4,2010-01-08,86.212303,86.249930,85.527544,85.700613,126402800


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [4]:
# Calculate EMA
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['EMA 20'] = [r.ema for r in indicators.get_ema(quotes, 20)]
df['EMA 25'] = [r.ema for r in indicators.get_ema(quotes, 25)]

# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima14'] = df['rsi'].rolling(14).mean()
df['rsi_bullish'] = 0.0
df['rsi_bullish'] = np.where(df['rsi'] > df['rsima14'], 1.0, 0.0)
df['crossover_rsi'] = df['rsi_bullish'].diff()

# Calculate ADX
df['ADX'] = [r.adx for r in indicators.get_adx(quotes, 14)]
df['DI_Plus'] = [r.pdi for r in indicators.get_adx(quotes, 14)]
df['DI_Minus'] = [r.mdi for r in indicators.get_adx(quotes, 14)]
df['adxr'] = [r.adxr for r in indicators.get_adx(quotes, 14)]

# Calculate EMA Crossover
df['ema_bullish'] = 0.0
df['ema_bullish'] = np.where(df['EMA 12'] > df['EMA 25'], 1.0, 0.0)
df['crossover_EMA12_EMA25'] = df['ema_bullish'].diff()

In [ ]:
if __name__ == '__main__':
    
    chart = Chart(title="EMA_RSI_ADX Strategy", maximize=True, inner_height=0.6)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart
    chart.set(df)

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_line=False, price_label=False)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_line=False, price_label=False)
    ema25_line.set(df[['Date', 'EMA 25']])

    # Create RSI subchart
    rsi_chart = chart.create_subchart(width=1.0,height=0.2, sync=True)
    rsi_line = rsi_chart.create_line('RSI', color="#3eff29")
    rsi_line.set(df[['Date', 'rsi']])
    rsima14_line = rsi_chart.create_line('rsima14', color="#bf2600")
    rsima14_line.set(df[['Date', 'rsima14']])
    
    # Create ADX subchart
    adx_chart = chart.create_subchart(width=1.0,height=0.2, sync=True)
    adx_line = adx_chart.create_line('ADX', color='#673ab7')
    adx_line.set(df[['Date', 'ADX']])
                  
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find trading signals
    for i in range(1, len(df)):
        current_time = df.iloc[i]['Date']
        
        # Check for potential signals based on EMA crossover
        if df.iloc[i]['crossover_EMA12_EMA25'] == 1:
            # Confirm with RSI and ADX conditions
            if (df.iloc[i]['rsi'] > 50 and  # RSI above 50 (uptrend momentum)
                df.iloc[i]['rsi'] < 70 and  # Not overbought
                df.iloc[i]['ADX'] > 25):    # Strong trend

                markers.append({
                    'time': current_time,
                    'position': 'below',
                    'shape': 'arrow_up',
                    'color': '#33de3d',
                    'text': 'Buy'
                })
        
        # Check for sell signals
        elif df.iloc[i]['crossover_EMA12_EMA25'] == -1:
            # Add sell marker when EMA crosses down
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once
    if markers:
        chart.marker_list(markers)
    
    chart.show(block=True)